# Data Cleaning/Processing

In [1]:
#Import all libraries

import pandas as pd
import os
import librosa
import numpy as np 
import matplotlib.pyplot as plt
import warnings

# Step 1: Import all data

CREMA-D only

In [3]:
Directory_name = r'C:\Users\Sarah Li\Documents\Masters of Data Analytics\DATA 608\Project\Audio Raw Data\AudioWAV'
Audio_file_paths = []

listings = os.listdir(Directory_name) #This gets all names of files in folder

i=0
for i in range(0,len(listings)):
    file = listings[i]
    filepath = f"{Directory_name}\{file}"
    Audio_file_paths.append(filepath)
    i+=1

#Reference:
#1) How to import multiple files at once: 
#https://stackoverflow.com/questions/39889680/loop-of-function-for-taking-multiple-audio-files-from-a-directory

In [4]:
#Check all paths have been saved properly
print(len(Audio_file_paths))
print(len(listings))

Audio_file_paths[:5]

7442
7442


['C:\\Users\\Sarah Li\\Documents\\Masters of Data Analytics\\DATA 608\\Project\\Audio Raw Data\\AudioWAV\\1001_DFA_ANG_XX.wav',
 'C:\\Users\\Sarah Li\\Documents\\Masters of Data Analytics\\DATA 608\\Project\\Audio Raw Data\\AudioWAV\\1001_DFA_DIS_XX.wav',
 'C:\\Users\\Sarah Li\\Documents\\Masters of Data Analytics\\DATA 608\\Project\\Audio Raw Data\\AudioWAV\\1001_DFA_FEA_XX.wav',
 'C:\\Users\\Sarah Li\\Documents\\Masters of Data Analytics\\DATA 608\\Project\\Audio Raw Data\\AudioWAV\\1001_DFA_HAP_XX.wav',
 'C:\\Users\\Sarah Li\\Documents\\Masters of Data Analytics\\DATA 608\\Project\\Audio Raw Data\\AudioWAV\\1001_DFA_NEU_XX.wav']

In [5]:
listings[:5]

#Reference:
#1) Getting the first few items from a list: https://www.geeksforgeeks.org/how-to-get-first-n-items-from-a-list-in-python/

['1001_DFA_ANG_XX.wav',
 '1001_DFA_DIS_XX.wav',
 '1001_DFA_FEA_XX.wav',
 '1001_DFA_HAP_XX.wav',
 '1001_DFA_NEU_XX.wav']

In [6]:
ID = [file[:4] for file in listings]
Phrase = [(file.split("_"))[1] for file in listings]
Emotion = [(file.split("_"))[2] for file in listings]
Level = [(file.split("_"))[3][:2] for file in listings]

In [7]:
F_ID = ['1002','1003','1004','1006','1007','1008','1009','1010','1012','1013','1018','1020','1021','1024','1025','1028',
        '1029','1030','1037','1043','1046','1047','1049','1052','1053','1054','1055','1056','1058','1060','1061','1063',
        '1072','1073','1074','1075','1076','1078','1079','1082','1084','1089','1091']
Gender=[]

for i in range(0,len(ID)):
    if ID[i] in F_ID:
        Gender.append('Female')
    else:
        Gender.append('Male')

#ID information from: https://www.kaggle.com/code/pranavbelhekar/audio-emotion-classification-part-1/notebook

In [8]:
#Seeing the outputs are as expected

print(ID[:5])
print(Phrase[:5])
print(Emotion[:5])
print(Level[:5])

['1001', '1001', '1001', '1001', '1001']
['DFA', 'DFA', 'DFA', 'DFA', 'DFA']
['ANG', 'DIS', 'FEA', 'HAP', 'NEU']
['XX', 'XX', 'XX', 'XX', 'XX']


In [9]:
identifiers=[ID,Gender,Phrase,Emotion,Level]
identifiers_df = pd.DataFrame(identifiers).transpose()
identifiers_df.columns = ['ID','Gender','Phrase','Emotion','Level']
display(identifiers_df)

,ID,Gender,Phrase,Emotion,Level
0,1001,Male,DFA,ANG,XX
1,1001,Male,DFA,DIS,XX
2,1001,Male,DFA,FEA,XX
3,1001,Male,DFA,HAP,XX
4,1001,Male,DFA,NEU,XX
...,...,...,...,...,...
7437,1091,Female,WSI,DIS,XX
7438,1091,Female,WSI,FEA,XX
7439,1091,Female,WSI,HAP,XX
7440,1091,Female,WSI,NEU,XX


# Step 2: Features Extraction

In [10]:
def feature_methods(path):
    data,sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # Zero Crossing  
    feature = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    feature=np.hstack((feature, zcr))

    # Chroma
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    feature = np.hstack((feature, chroma_stft))

    # MFCC Base
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    feature = np.hstack((feature, mfcc))

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    feature = np.hstack((feature, rms))

    # Mel Spectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    feature = np.hstack((feature, mel)) 

    #Spectral Contrast
    spectral=np.mean(librosa.feature.spectral_contrast(y=data, sr=sample_rate).T, axis=0)
    feature= np.hstack((feature,spectral))
    
    return feature

In [11]:
features = []

i=0
for i in range (0,len(Audio_file_paths)):
    finalfeatures = feature_methods(Audio_file_paths[i])
    features.append(finalfeatures)
    i+=1

C:\Users\Sarah Li\anaconda3\lib\site-packages\librosa\core\pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [12]:
feature_df=pd.DataFrame({"Feature":features})
FeatureV = pd.concat([identifiers_df,pd.DataFrame(feature_df['Feature'].values.tolist())],axis=1)
FeatureV = FeatureV.fillna(0)
FeatureV = FeatureV[FeatureV.Emotion != "NEU"]
display(FeatureV)

,ID,Gender,Phrase,Emotion,Level,0,1,2,3,4,...,159,160,161,162,163,164,165,166,167,168
0,1001,Male,DFA,ANG,XX,0.085690,0.671934,0.694386,0.672386,0.675260,...,4.833769e-08,4.774562e-08,4.738036e-08,20.236447,12.717658,17.725536,16.395970,19.851038,18.746176,64.114904
1,1001,Male,DFA,DIS,XX,0.078444,0.718596,0.729192,0.653268,0.665332,...,3.278495e-07,3.231356e-07,3.202182e-07,19.544465,13.033411,16.184696,14.940196,18.819783,17.493382,65.144855
2,1001,Male,DFA,FEA,XX,0.071806,0.576044,0.606729,0.631089,0.654011,...,2.048362e-06,2.016054e-06,1.996248e-06,20.332696,15.057014,17.835584,16.478764,19.712501,17.948928,63.113899
3,1001,Male,DFA,HAP,XX,0.069309,0.666179,0.677285,0.701381,0.676417,...,1.185680e-07,1.169627e-07,1.159695e-07,19.320487,14.540849,16.938015,15.095720,19.876366,17.662157,62.585219
5,1001,Male,DFA,SAD,XX,0.069696,0.697295,0.710469,0.657157,0.680874,...,6.190247e-08,6.069696e-08,5.997680e-08,20.679260,12.612233,16.869543,15.638160,19.075156,17.443968,62.328887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7436,1091,Female,WSI,ANG,XX,0.114518,0.587432,0.625285,0.559025,0.530899,...,1.455467e-08,1.393272e-08,1.356538e-08,17.970945,15.397158,16.427824,16.965998,17.989314,17.473032,68.055226
7437,1091,Female,WSI,DIS,XX,0.066478,0.591427,0.572732,0.558411,0.564348,...,1.854230e-09,1.844318e-09,1.837963e-09,16.775219,14.104081,15.905302,16.261985,17.625420,16.952061,67.380123
7438,1091,Female,WSI,FEA,XX,0.087736,0.515751,0.512463,0.514055,0.513658,...,3.920627e-10,3.877069e-10,3.850097e-10,18.002724,15.550379,16.384595,15.627693,17.630192,17.117918,68.745353
7439,1091,Female,WSI,HAP,XX,0.129660,0.434935,0.465356,0.475787,0.471286,...,5.890358e-09,5.813106e-09,5.765292e-09,17.159134,14.972380,17.922721,16.988440,18.537291,18.036333,66.545819


In [13]:
FeatureV.to_csv(r'C:\Users\Sarah Li\Documents\Masters of Data Analytics\DATA 608\Project\FeatureV.csv', index=False)